In [2]:
import pandas as pd
import numpy as np
from modules import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt
from nltk.util import everygrams

In [3]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘'
                 ,'최고','느리다','마을','최고','항상','포장','재다','말다','목','넘김','만족하다','정말','역시','아주','감사하다','오다','자주','요','늘','넘다','않다','더','병','다','숙성','나다','술','드리다','마트','꼼꼼하다','잇다'
                 ,'복','순도','용','비싸다','가격','많다','강하다','보내다','추천','보고','쏘다','느낌','사다','처음','많이','톡','생각','엄청','날','아니다'
                 ,'믿다', '구입', '사먹다', '한번', '싶다', '콤', '완전', '제품', '재', '해주다', '말', '사람']) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [4]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
name_list =["술 리뷰(곰표막걸리)","술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]

In [5]:
all_data = pd.DataFrame()

for name in name_list:
    data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    try:
        data.columns = ['date', 'star','doc']
    except:
        data.columns = ['date', 'star','doc','name']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


C:\Users\user\AppData\Local\Temp\ipykernel_13716\3516534953.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\3516534953.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\3516534953.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\3516534953.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\3516534953.py:11: FutureWarning

In [6]:
all_data

,date,star,doc,name
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...,NaN
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...,NaN
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...,NaN
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...,NaN
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요,NaN
...,...,...,...,...
5674,2020-04-03,5,다양한술을 집에서 편하게 주문가능하기에 자주 주문합니다이번에도 즐겁게 마시겠읍니다,NaN
5675,2020-04-03,5,빠른 배송 감사합니당,NaN
5676,2020-03-31,5,단맛은 있는데 꿀 단맛이어서 나쁘지 않아요 기분 나쁘지 않은 단맛이에요 인공감미료가...,NaN
5677,2020-03-26,5,맛이 달고 청량감이 좋아요,NaN


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# 특징을 추출할 문서의 tf-idf를 구함
# "술 리뷰(곰표막걸리)","술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"
bow_rep_tfidf = tfidf_fit.transform(globals()["술 리뷰(복순도가)"].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})



c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출
word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30]

C:\Users\user\AppData\Local\Temp\ipykernel_13716\3393798343.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30]


,단어,tf-idf,idf
5456,샴페인,248.509309,5.501938
4765,부모님,242.848549,5.122699
113,가족,189.790126,5.283560
9256,지인,187.419217,5.068860
9944,친구,181.900120,5.133604
2168,달달,172.155863,4.140109
3748,명절,167.085410,5.783826
1302,깔끔하다,141.364156,4.423065
7351,와인,137.015262,5.828947
6031,스파클링,129.508419,6.132133


In [10]:
word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30].to_csv('복순도가_특징.csv', encoding='utf-8-sig')

C:\Users\user\AppData\Local\Temp\ipykernel_13716\4293238168.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.9][word_count['tf-idf']>30].to_csv('복순도가_특징.csv', encoding='utf-8-sig')


In [11]:
df = pd.read_csv('복순도가_특징.csv', encoding='utf-8-sig')
del df['Unnamed: 0']
df

,단어,tf-idf,idf
0,샴페인,248.509309,5.501938
1,부모님,242.848549,5.122699
2,가족,189.790126,5.283560
3,지인,187.419217,5.068860
4,친구,181.900120,5.133604
5,달달,172.155863,4.140109
6,명절,167.085410,5.783826
7,깔끔하다,141.364156,4.423065
8,와인,137.015262,5.828947
9,스파클링,129.508419,6.132133


In [28]:
Tags = {
	"common": [
		"tech", "technology", "computer", "computerScience", "gizmo", 
		"geek", "strictly_for_geeks", "stuff", "swag", "disruption", "programming", 
		"coding", "sourceCode", "entrepreneur", "entrepreneurship", 
		"developer", "developers", "software", "development", "softwareDeveloper"
	],

	"android": [
		"android", "smartPhone", "androidStudio", "studio", "java", "mobile",
		"mobileApplication", "mobileDeveloper", "google", "googleDevelopers", 
		"androidDevelopent", "UX", "UI"
	],

	"web": [
		"web", "webDevelopment", "frontEnd", "backEnd", "website", "webapp",
		"cloud", "hosting", "webDeveloper", "database", "html", "css", "javascript",
		"html5", "js", "webdesign", "webdesigner", "internet", "IT", "data", "UX", "UI"
	],

	"raspberrypi": [
		"raspberrypi", "pi", "hardware", "microcontroller", "IoT", "automation"
	]
}

In [29]:
# from tagsList import Tags
def getTags(list_of_keywords):
	tags = Tags['common']

	for key in list_of_keywords:
		for tag in Tags[key]:
			tags.append(tag)

	return list(set(tags))

def getHashTagString(list_of_tags):
	hashTagString = ""

	for tag in list_of_tags:
		hashTagString += " #" + tag

	return hashTagString

In [30]:
Tags['common']

['tech',
 'technology',
 'computer',
 'computerScience',
 'gizmo',
 'geek',
 'strictly_for_geeks',
 'stuff',
 'swag',
 'disruption',
 'programming',
 'coding',
 'sourceCode',
 'entrepreneur',
 'entrepreneurship',
 'developer',
 'developers',
 'software',
 'development',
 'softwareDeveloper']

In [31]:
for i in df['단어']:
    Tags['common'].append(i)
Tags['common']

['tech',
 'technology',
 'computer',
 'computerScience',
 'gizmo',
 'geek',
 'strictly_for_geeks',
 'stuff',
 'swag',
 'disruption',
 'programming',
 'coding',
 'sourceCode',
 'entrepreneur',
 'entrepreneurship',
 'developer',
 'developers',
 'software',
 'development',
 'softwareDeveloper',
 '샴페인',
 '부모님',
 '가족',
 '지인',
 '친구',
 '달달',
 '명절',
 '깔끔하다',
 '와인',
 '스파클링',
 '시큼하다',
 '시원하다',
 '계속',
 '추석',
 '생각나다',
 '달다',
 '신맛',
 '고급',
 '상큼',
 '빨르다',
 '최애',
 '언제나',
 '괜찮다',
 '두번째',
 '달',
 '기대',
 '청량감',
 '전',
 '고급스럽다',
 '인생',
 '특별하다']

In [35]:
import sys
print(getHashTagString(getTags(sys.argv[1:])).strip())

KeyError: '--ip=127.0.0.1'